In [20]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [21]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [22]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [23]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

100%|██████████| 1/1 [00:09<00:00,  9.75s/it]


,Id,Empresa__r.Id,Empresa__r.Name,Empresa__r.Holding__c,Empresa__r.Campa_a__c,Empresa__r.Id_Empresa__c,Empresa__r.Empleador__c,Empresa__r.poliza__c,Empresa__r.RUT_Empresa__c,Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c,Name
0,a1W8c000007c5aIEAQ,0018c00002k9KwBAAU,Aldo Conti MX,-,Aldo Conti,EMP10376,Aldo Conti MX,-,-,-,False,A-9930
1,a1W8c000007c669EAA,0018c00002k9yOCAAY,Caja de Compensación de Asignación Familiar de...,Itaú,Vida Security,EMP10377,Caja de Compensación de Asignación Familiar de...,283144,81826800-9,-,False,A-9931
2,a1W8c000007c66AEAQ,0018c00002k9yODAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10378,Itaú Corpbanca,283147,97023000-9,-,False,A-9932
3,a1W8c000007c66BEAQ,0018c00002k9yOEAAY,Itaú Corpbanca,Itaú - sin servicio,Vida Security,EMP10379,Itaú Corpbanca,283148,-,-,False,A-9933
4,a1W8c000007c66CEAQ,0018c00002k9yOFAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10380,Itaú Corpbanca,283149,-,-,False,A-9934


vllamadaEmpresa.shape


In [24]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(8285, 8)

In [25]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7913, 5)

In [26]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(8291, 10)

In [27]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [28]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:09<00:00,  9.12s/it]


,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,False,a3yRN0000006mmjYAA,775173998Sin Póliza Asociada,1
1,-,Apunto Servicios De Alimentacion,Apunto Servicios De Alimentacion,-,Consalud,Holding Consalud,-,Chile,False,a3yRN0000006rG9YAI,-,1
2,-,2M Inmobiliaria Spa.,2M Inmobiliaria Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGAYAY,-,1
3,-,360 Gms Capacitacion Spa.,360 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGBYAY,-,1
4,-,361 Gms Capacitacion Spa.,361 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGCYAY,-,1


fin empresas y comienzo de rescate de pacientes


In [29]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId": "Id Contacto",
    }
)
Pacientes_SF.shape

(1935433, 10)

In [30]:
def siPrimerCero(y):
    if len(y) <= 4:
        return "NONE"
    if y[0] == "0":
        y = y[1:]
    return y

In [31]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\928634061.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")


In [32]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE")]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE")]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas\_\_c relaciones de empresas con pacientes

results = sf.query_all(
"""
select Id,Paciente**c, Programas_por_Empresas**c,padre**c from Paciente_Programas**c  
 """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
columns={
"Id": "Id Pacientes_Programas**c",
"Paciente**c": "Id Account",
"Programas_por_Empresas**c": "Programas_por_Empresas**c",
"padre\_\_c": "Nombre Campaña",
}
)
Paciente_Programas_SF.shape


# Union de Pacientes Programas y Programas por empresa

empresas_unidas = pd.merge(
left=Paciente_Programas_SF,
right=Programas_empresas,
how="left",
left_on="Programas_por_Empresas**c",
right_on="Programas_por_Empresas**c",
)
empresas_unidas.shape


Carga de Archivo carga


In [85]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife Agencia"
Pais = "Chile"
baseFinal = True
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,poliza,rutE,empleador,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,rut1,...,direccion,comuna,ciudad,celular,email,nacimiento,sexo,VIGENCIA,VIGENCIADESDE,VIGENCIAHASTA
0,123240000001,B,MetLife Joven Premium,GERVACIO AARON,RODRIGUEZ,PEÑA,RODRIGUEZ PEÑA,19333177,7,19333177-7,...,Valle Trancura 1235,CONCEPCION,CONCEPCION,57658381,rodriguezgervacio@gmail.com,1996-05-01,M,VIGENTE,2024-01-04,2025-01-04
1,123240000001,B,MetLife Joven Premium,CATALINA ANDREA,MANRÍQUEZ,NÚÑEZ,MANRÍQUEZ NÚÑEZ,20197475,5,20197475-5,...,Valle Trancura 1235,CONCEPCION,CONCEPCION,975171632,cata.manriquezn@gmail.com,2000-04-07,F,VIGENTE,2024-01-04,2025-01-04
2,123240000002,B,MetLife Joven Premium,MAURICIO JAIME,HIDALGO,VELASQUEZ,HIDALGO VELASQUEZ,15441551,3,15441551-3,...,Camino Al Volcan/ Las Viscachas 05879 Cod 3/ C-9,PUENTE ALTO,SANTIAGO,89019246,MAURICIO.HIDALGO.V@GMAIL.COM,1982-09-14,M,VIGENTE,2024-01-04,2025-01-04
3,123240000003,B,MetLife Joven,MARCIA ISABEL,SOTO,ANTILEF,SOTO ANTILEF,18848699,1,18848699-1,...,Martinez De Rosas 0362,LONCOCHE,LONCOCHE,84557671,flga.msoto@gmail.com,1994-09-23,F,VIGENTE,2024-01-04,2025-01-04
4,123240000004,B,MetLife Joven,MARIA JOSE,SALINAS,CAMPUSANO,SALINAS CAMPUSANO,16557810,4,16557810-4,...,Psj El Tulipero 3690,MACUL,SANTIAGO,8888081,salinasc.majo@gmail.com,1987-07-09,F,VIGENTE,2024-01-04,2025-01-04


In [86]:
Pacientes_Archivo.shape

(421, 21)

In [87]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",
        "Id Contacto",
    ]
]

In [88]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("-", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("–", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(" ", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.upper()
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.strip()
Pacientes_Archivo.shape

Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].astype(str)
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.upper()
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("-", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("–", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\1490231206.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\1490231206.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\1490231206.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["ru

(421, 22)

In [89]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\2296516763.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(421, 23)

In [90]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┐", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("═", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("╔", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┌", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┐", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("═", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("╔", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┌", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\3410721566.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\3410721566.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [91]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo

# Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
# Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

,poliza,rutE,empleador,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,rut1,...,ciudad,celular,email,nacimiento,sexo,VIGENCIA,VIGENCIADESDE,VIGENCIAHASTA,rut,rutT
0,123240000001,B,MetLife Joven Premium,Gervacio Aaron,RODRIGUEZ,PEÑA,Rodriguez Peña,19333177,7,19333177-7,...,CONCEPCION,57658381,rodriguezgervacio@gmail.com,1996-05-01,M,VIGENTE,2024-01-04,2025-01-04,193331777,193331777
1,123240000001,B,MetLife Joven Premium,Catalina Andrea,MANRÍQUEZ,NÚÑEZ,Manríquez Núñez,20197475,5,20197475-5,...,CONCEPCION,975171632,cata.manriquezn@gmail.com,2000-04-07,F,VIGENTE,2024-01-04,2025-01-04,201974755,201974755
2,123240000002,B,MetLife Joven Premium,Mauricio Jaime,HIDALGO,VELASQUEZ,Hidalgo Velasquez,15441551,3,15441551-3,...,SANTIAGO,89019246,MAURICIO.HIDALGO.V@GMAIL.COM,1982-09-14,M,VIGENTE,2024-01-04,2025-01-04,154415513,154415513
3,123240000003,B,MetLife Joven,Marcia Isabel,SOTO,ANTILEF,Soto Antilef,18848699,1,18848699-1,...,LONCOCHE,84557671,flga.msoto@gmail.com,1994-09-23,F,VIGENTE,2024-01-04,2025-01-04,188486991,188486991
4,123240000004,B,MetLife Joven,Maria Jose,SALINAS,CAMPUSANO,Salinas Campusano,16557810,4,16557810-4,...,SANTIAGO,8888081,salinasc.majo@gmail.com,1987-07-09,F,VIGENTE,2024-01-04,2025-01-04,165578104,165578104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,123240000384,B,MetLife Joven,Marco Antonio,SOBARZO,MALDONADO,Sobarzo Maldonado,15356614,3,15356614-3,...,SANTIAGO,81961397,mmaldonado@mutucar.cl,1982-08-06,M,VIGENTE,2024-02-12,2025-02-12,153566143,153566143
417,123240000385,B,MetLife Joven Premium,Cristian Gaston,FLORES,ESPINOZA,Flores Espinoza,12540305,0,12540305-0,...,PUERTO MONTT,78998415,cristian.flores@andeslogistics.cl,1973-12-10,M,VIGENTE,2024-02-12,2025-02-12,125403050,125403050
418,123240000385,B,MetLife Joven Premium,Martina Ignacia,FLORES,HERNANDEZ,Flores Hernandez,23199861,6,23199861-6,...,0,0,0,2009-12-15,F,VIGENTE,2024-02-12,2025-02-12,231998616,231998616
419,123240000386,B,MetLife Joven,Juan Ignacio Ricardo,PEDRAZA,PARADA,Pedraza Parada,18167650,7,18167650-7,...,SANTIAGO,66226845,J.IGNACIO.PEDRAZA@GMAIL.COM,1992-05-18,M,VIGENTE,2024-02-12,2025-02-12,181676507,181676507


In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False)

In [92]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE")]
# | (Pacientes_Archivo["rutT"] != "NONE")


In [ ]:
Pacientes_Archivo["Llave"] = Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]
Pacientes_Archivo["PolizaLlave"] = Pacientes_Archivo["poliza"] +  Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]

Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/eliminar2.xlsx", index=False)

PAcientes Programas


In [93]:
query_pacientePrograma = (
    "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '"
    + campaña
    + "'"
)
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()

100%|██████████| 1/1 [00:08<00:00,  8.62s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id
0,001RN000000qmWpYAI,001RN000000mbV8YAI,243083125,a1YRN0000002pWH2AY
1,001RN000000qmWqYAI,001RN000000mbV8YAI,190689700,a1YRN0000002pWI2AY
2,0018c00002eiMlWAAU,001RN000000mbV8YAI,128466312,a1YRN0000002pWJ2AY
3,001RN000000qmWrYAI,001RN000000mbV8YAI,9908331K,a1YRN0000002pWK2AY
4,001RN000000qmWsYAI,001RN000000mbV8YAI,106821992,a1YRN0000002pWL2AY


In [94]:
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()

,Id Account Paciente,Id Account Empleador,Rut,Id Paciente Programas
0,001RN000000qmWpYAI,001RN000000mbV8YAI,243083125,a1YRN0000002pWH2AY
1,001RN000000qmWqYAI,001RN000000mbV8YAI,190689700,a1YRN0000002pWI2AY
2,0018c00002eiMlWAAU,001RN000000mbV8YAI,128466312,a1YRN0000002pWJ2AY
3,001RN000000qmWrYAI,001RN000000mbV8YAI,9908331K,a1YRN0000002pWK2AY
4,001RN000000qmWsYAI,001RN000000mbV8YAI,106821992,a1YRN0000002pWL2AY


In [95]:
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\3172767859.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")


(3394, 4)

In [96]:
PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head()

,Id Account Paciente,Id Account Empleador,Rut_x,Id Paciente Programas,Rut_y,Id Contacto
0,001RN000000qmWpYAI,001RN000000mbV8YAI,243083125,a1YRN0000002pWH2AY,243083125,003RN000002OEwUYAW
1,001RN000000qmWqYAI,001RN000000mbV8YAI,190689700,a1YRN0000002pWI2AY,190689700,003RN000002OEwVYAW
2,0018c00002eiMlWAAU,001RN000000mbV8YAI,128466312,a1YRN0000002pWJ2AY,128466312,0038c000039ILvZAAW
3,001RN000000qmWrYAI,001RN000000mbV8YAI,9908331K,a1YRN0000002pWK2AY,9908331K,003RN000002OEwWYAW
4,001RN000000qmWsYAI,001RN000000mbV8YAI,106821992,a1YRN0000002pWL2AY,106821992,003RN000002OEwXYAW


account contact relation


In [97]:
query_accountContact = (
    "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '"
    + campaña
    + "'"
)
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()

100%|██████████| 1/1 [00:04<00:00,  4.80s/it]


,Id,ContactId,AccountId,RutContact__c,Contact.AccountId
0,07kRN000000KW8cYAG,0038c000039I4yoAAC,001RN000000mbV8YAI,270995241,0018c00002ei5rNAAQ
1,07kRN000000KW8dYAG,003RN000002OJvRYAW,001RN000000mbV8YAI,129138831,001RN000000qp8bYAA
2,07kRN000000KW8eYAG,003RN000002OJvSYAW,001RN000000mbV8YAI,14384319K,001RN000000qp8cYAA
3,07kRN000000KW8fYAG,003RN000002OJvTYAW,001RN000000mbV8YAI,78177799,001RN000000qp8dYAA
4,07kRN000000KW8gYAG,003RN000002OJvUYAW,001RN000000mbV8YAI,114276863,001RN000000qp8eYAA


In [98]:
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId": "Id Account Paciente",
    }
)
accountContactRelation.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut,Id Account Paciente
0,07kRN000000KW8cYAG,0038c000039I4yoAAC,001RN000000mbV8YAI,270995241,0018c00002ei5rNAAQ
1,07kRN000000KW8dYAG,003RN000002OJvRYAW,001RN000000mbV8YAI,129138831,001RN000000qp8bYAA
2,07kRN000000KW8eYAG,003RN000002OJvSYAW,001RN000000mbV8YAI,14384319K,001RN000000qp8cYAA
3,07kRN000000KW8fYAG,003RN000002OJvTYAW,001RN000000mbV8YAI,78177799,001RN000000qp8dYAA
4,07kRN000000KW8gYAG,003RN000002OJvUYAW,001RN000000mbV8YAI,114276863,001RN000000qp8eYAA


In [99]:
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\2268980097.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")


(3404, 5)

In [100]:
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut_x,Id Account Paciente,Rut_y,Id Contacto
0,07kRN000000KW8cYAG,0038c000039I4yoAAC,001RN000000mbV8YAI,270995241,0018c00002ei5rNAAQ,270995241,0038c000039I4yoAAC
1,07kRN000000KW8dYAG,003RN000002OJvRYAW,001RN000000mbV8YAI,129138831,001RN000000qp8bYAA,129138831,003RN000002OJvRYAW
2,07kRN000000KW8eYAG,003RN000002OJvSYAW,001RN000000mbV8YAI,14384319K,001RN000000qp8cYAA,14384319K,003RN000002OJvSYAW
3,07kRN000000KW8fYAG,003RN000002OJvTYAW,001RN000000mbV8YAI,78177799,001RN000000qp8dYAA,78177799,003RN000002OJvTYAW
4,07kRN000000KW8gYAG,003RN000002OJvUYAW,001RN000000mbV8YAI,114276863,001RN000000qp8eYAA,114276863,003RN000002OJvUYAW


empresa contacto convenio


In [101]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc, Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c, Empresa__c, Paciente__c,N_Documento_Titular__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [00:12<00:00, 12.67s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__c,Paciente__c,N_Documento_Titular__c
0,a40RN0000019a6vYAA,132416842,-,-,a3yRN0000006tT9YAI,001RN000000qos9YAA,132416842
1,a40RN0000019ex8YAA,257966194,-,-,a3yRN0000006tT9YAI,001RN0000069HW1YAM,257966194


In [102]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c": "Rut Titular",
        "Empresa__r.P_liza__c":"Poliza",
        "Empresa__r.Rut_Empresa__c":"Rut Empresa",
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular
0,a40RN0000019a6vYAA,132416842,-,-,a3yRN0000006tT9YAI,001RN000000qos9YAA,132416842
1,a40RN0000019ex8YAA,257966194,-,-,a3yRN0000006tT9YAI,001RN0000069HW1YAM,257966194


In [103]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(" ", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()

empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("-", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("–", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(" ", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.upper()
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.strip()

empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("-", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("–", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(".", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(" ", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.upper()
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.strip()


empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\3330564967.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\3330564967.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\3330564967.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when r

(2, 7)

In [104]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Rut_y,Id Contacto
0,a40RN0000019a6vYAA,132416842,,,a3yRN0000006tT9YAI,001RN000000qos9YAA,132416842,132416842,003RN000002OJN8YAO
1,a40RN0000019ex8YAA,257966194,,,a3yRN0000006tT9YAI,001RN0000069HW1YAM,257966194,257966194,003RN00000J0IlBYAV


In [ ]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

In [ ]:
empresacontactoconvenio_PacientesSF.shape

Rescatamos todos los ruts Id con su empresa desde la campaña


In [105]:
Pacientes_Archivo.head()

,poliza,rutE,empleador,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,rut1,...,ciudad,celular,email,nacimiento,sexo,VIGENCIA,VIGENCIADESDE,VIGENCIAHASTA,rut,rutT
0,123240000001,B,MetLife Joven Premium,Gervacio Aaron,RODRIGUEZ,PEÑA,Rodriguez Peña,19333177,7,19333177-7,...,CONCEPCION,57658381,rodriguezgervacio@gmail.com,1996-05-01,M,VIGENTE,2024-01-04,2025-01-04,193331777,193331777
1,123240000001,B,MetLife Joven Premium,Catalina Andrea,MANRÍQUEZ,NÚÑEZ,Manríquez Núñez,20197475,5,20197475-5,...,CONCEPCION,975171632,cata.manriquezn@gmail.com,2000-04-07,F,VIGENTE,2024-01-04,2025-01-04,201974755,201974755
2,123240000002,B,MetLife Joven Premium,Mauricio Jaime,HIDALGO,VELASQUEZ,Hidalgo Velasquez,15441551,3,15441551-3,...,SANTIAGO,89019246,MAURICIO.HIDALGO.V@GMAIL.COM,1982-09-14,M,VIGENTE,2024-01-04,2025-01-04,154415513,154415513
3,123240000003,B,MetLife Joven,Marcia Isabel,SOTO,ANTILEF,Soto Antilef,18848699,1,18848699-1,...,LONCOCHE,84557671,flga.msoto@gmail.com,1994-09-23,F,VIGENTE,2024-01-04,2025-01-04,188486991,188486991
4,123240000004,B,MetLife Joven,Maria Jose,SALINAS,CAMPUSANO,Salinas Campusano,16557810,4,16557810-4,...,SANTIAGO,8888081,salinasc.majo@gmail.com,1987-07-09,F,VIGENTE,2024-01-04,2025-01-04,165578104,165578104


Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)


Pacientes_Archivo.drop_duplicates(
[
"rut",
"rutT",
"LLaveEmpresa",
"LLAveRut",
],
keep="last",
)
Pacientes_Archivo.shape


In [106]:
print(campaña)

MetLife Agencia


In [107]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [108]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(421, 34)

In [109]:
Pacientes_con_id.head()

,poliza,rutE,empleador,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,rut1,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
0,123240000001,B,MetLife Joven Premium,Gervacio Aaron,RODRIGUEZ,PEÑA,Rodriguez Peña,19333177,7,19333177-7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,123240000001,B,MetLife Joven Premium,Catalina Andrea,MANRÍQUEZ,NÚÑEZ,Manríquez Núñez,20197475,5,20197475-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123240000002,B,MetLife Joven Premium,Mauricio Jaime,HIDALGO,VELASQUEZ,Hidalgo Velasquez,15441551,3,15441551-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123240000003,B,MetLife Joven,Marcia Isabel,SOTO,ANTILEF,Soto Antilef,18848699,1,18848699-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,123240000004,B,MetLife Joven,Maria Jose,SALINAS,CAMPUSANO,Salinas Campusano,16557810,4,16557810-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [110]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        print("date_string =", date_string)
        print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        print("date_object =", date_object)
        print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    

In [ ]:
Pacientes_con_id["dertyuiop"] = Pacientes_con_id["nacimiento"]
Pacientes_con_id["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
Pacientes_con_id.head()

In [111]:


# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# Subir_pacientes["Email_Seteado__c"] = ""
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )


if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
#if ("nacimiento" in Pacientes_con_id.columns) == True:    
#    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"].astype(str), format="%Y-%m-%d")  
if ("nacimiento" in Pacientes_con_id.columns) == True:    
    #try:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    Subir_pacientes["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    #except ValueError:
        #Subir_pacientes["PersonBirthdate"] = pd.to_datetime("1900-01-01", format="%Y-%m-%d")
 
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"

Subir_pacientes_Que_no_Estan = Subir_pacientes
Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]

#Subir_pacientes["PersonBirthdate"] = Subir_pacientes["PersonBirthdate"].str.replace("0001-01-01 00:00:00", "")

# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)
Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
)
Subir_pacientes.head()

date_string = 1996-05-01 00:00:00
type of date_string = <class 'str'>
date_object = 1996-05-01 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 2000-04-07 00:00:00
type of date_string = <class 'str'>
date_object = 2000-04-07 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1982-09-14 00:00:00
type of date_string = <class 'str'>
date_object = 1982-09-14 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1994-09-23 00:00:00
type of date_string = <class 'str'>
date_object = 1994-09-23 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1987-07-09 00:00:00
type of date_string = <class 'str'>
date_object = 1987-07-09 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1982-08-10 00:00:00
type of date_string = <class 'str'>
date_object = 1982-08-10 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1991-04-25 00:00:00
type of date_string = <class 'str'>
date_o

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\179516991.py:94: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\179516991.py:101: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
0,False,Gervacio Aaron,Rodriguez Peña,193331777,rodriguezgervacio@gmail.com,True,1996-05-01,1996-05-01,Hombre,valle trancura 1235,Concepcion,Concepcion,57658381,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1,False,Catalina Andrea,Manríquez Núñez,201974755,cata.manriquezn@gmail.com,True,2000-04-07,2000-04-07,Mujer,valle trancura 1235,Concepcion,Concepcion,975171632,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
2,False,Mauricio Jaime,Hidalgo Velasquez,154415513,mauricio.hidalgo.v@gmail.com,True,1982-09-14,1982-09-14,Hombre,camino al volcan/ las viscachas 05879 cod 3/ c-9,Puente Alto,Santiago,8919246,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3,False,Marcia Isabel,Soto Antilef,188486991,flga.msoto@gmail.com,True,1994-09-23,1994-09-23,Mujer,martinez de rosas 0362,Loncoche,Loncoche,84557671,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
4,False,Maria Jose,Salinas Campusano,165578104,salinasc.majo@gmail.com,True,1987-07-09,1987-07-09,Mujer,psj el tulipero 3690,Macul,Santiago,888881,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [112]:
Subir_pacientes.shape

(296, 18)

In [113]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc where IdentificationId__pc = '" + rut + "')"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    #vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9] ,
                        #"PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

' sf_res =None\ntry:\n    # Create the new Contact record in Salesforce\n    sf_res = sf.Contact.create(new_contact)\nexcept SalesforceMalformedRequest as e:\n    if \'duplicateResut\' in e[0]:\n        print(f"Malformed request error: {e}")\n        # Handle the error here as needed\n        #Bypass SF Email duplicate rule by removing the email to create a SF Lead\n        del new_contact[\'Email\']\n        sf_res = sf.Contact.create(new_contact)\n\n\nif sf_res is not None:\n    # Verify that the Contact was created successfully\n    created_contact = sf.Contact.get(sf_res[\'id\'])\n    print(created_contact) '

In [114]:
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
0,False,Gervacio Aaron,Rodriguez Peña,193331777,rodriguezgervacio@gmail.com,True,1996-05-01,1996-05-01,Hombre,valle trancura 1235,Concepcion,Concepcion,57658381,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1,False,Catalina Andrea,Manríquez Núñez,201974755,cata.manriquezn@gmail.com,True,2000-04-07,2000-04-07,Mujer,valle trancura 1235,Concepcion,Concepcion,975171632,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
2,False,Mauricio Jaime,Hidalgo Velasquez,154415513,mauricio.hidalgo.v@gmail.com,True,1982-09-14,1982-09-14,Hombre,camino al volcan/ las viscachas 05879 cod 3/ c-9,Puente Alto,Santiago,8919246,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3,False,Marcia Isabel,Soto Antilef,188486991,flga.msoto@gmail.com,True,1994-09-23,1994-09-23,Mujer,martinez de rosas 0362,Loncoche,Loncoche,84557671,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
4,False,Maria Jose,Salinas Campusano,165578104,salinasc.majo@gmail.com,True,1987-07-09,1987-07-09,Mujer,psj el tulipero 3690,Macul,Santiago,888881,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [115]:
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)

Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)

Subir_pacientes.shape

vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vaci

(296, 19)

In [116]:
Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]
# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()
Subir_pacientes.shape

(296, 19)

In [ ]:
#Subir_pacientes["dato_account"].head()

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/Todesffff.xlsx", index=False)

In [117]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [118]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c WHERE Empresa_Padre__r.Name = '" + campaña + "'"

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]

df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_asociadas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:02<00:00,  2.67s/it]


,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,-,MetLife A.,MetLife A.,-,MetLife Agencia,-,MetLife Orienta,Chile,True,a3yRN0000006tT9YAI,-,1
1,-,MetLife Producto Joven,MetLife Producto Joven,-,MetLife Agencia,-,MetLife Orienta,Chile,True,a3yRN0000009psvYAA,-,1


In [119]:
df2.shape

(2, 12)

In [ ]:
Subir_empresacontactoconvenio.head()

In [122]:
Pacientes_con_id_no = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == False]
Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
    lambda y: traerId(y)
)
Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
    lambda y: traerContact(y)
)
Pacientes_con_id_no.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_no.xlsx", index=False
)

Pacientes_con_id_si = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == True]
Pacientes_con_id_si.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_si.xlsx", index=False
)

Pacientes_con_id_final = pd.concat(
    [
        Pacientes_con_id_si,
        Pacientes_con_id_no,
    ],
    axis=0,
)

Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final.xlsx", index=False
)

if campaña == "Southbridge Plan Mujer":
    VpacientePrograma = "a1WRN0000007YwD2AU"
    VempresaaAccount = "001RN000003rwVaYAI"
    VempresaConvenio = "a3yRN0000006sVTYAY"
elif campaña == "Southbridge":
    VpacientePrograma = "a1W8c0000083pM9EAI"
    VempresaaAccount = "0018c00002f3Ev2AAE"
    VempresaConvenio = "a3yRN0000006sqYYAQ"
elif campaña == "Construmart":
    VpacientePrograma = "a1W8c0000083q9TEAQ"
    VempresaaAccount = "0018c00002ghJgBAAU"
    VempresaConvenio = "a3yRN0000006rlZYAQ"
elif campaña == "EY":
    VpacientePrograma = "a1W8c0000083laZEAQ"
    VempresaaAccount = "0018c00002eTboVAAS"
    VempresaConvenio = "a3yRN0000006rsTYAQ"
elif campaña == "Colmena":
    VpacientePrograma = "a1W8c0000083j8rEAA"
    VempresaaAccount = "0018c00002cXclQAAS"
    VempresaConvenio = "a3yRN0000006rf5YAA"
elif campaña == "Pedro De Valdivia":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sVzYAI"
elif campaña == "Forum":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s3SYAQ"
elif campaña == "Cruz Verde":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rsqYAA"
elif campaña == "Milan Fajanovic":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sR4YAI"
elif campaña == "ProChile":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rpsYAA"
elif campaña == "Farmacias Ahumada":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rspYAA"
elif campaña == "Minera El Abra":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006snOYAQ"
elif campaña == "Instruvalve":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s9DYAQ"
elif campaña == "VidaCel":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006suzYAA"
elif campaña == "Care Assistance Colaboradores":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006reXYA"
elif campaña == "Nexo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sORYAY"
elif campaña == "Talo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006t10YAA"
else:
    VpacientePrograma = "Buscarv"
    VempresaaAccount = "Buscarv"
    VempresaConvenio = "Buscarv"

#Programas por empresas
Pacientes_con_id_final["esonoes"] = Pacientes_con_id_final["rut"].isin(
    PacientesProgramas_PacientesSF["Rut_y"]
)
""" Subir_programas_empresa = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes"] == False
]
Subir_programas_empresa["Paciente__c"] = Subir_programas_empresa["Id Account Paciente"]
Subir_programas_empresa["Programas_por_Empresas__c"] = VpacientePrograma
Subir_programas_empresa = Subir_programas_empresa[
    [
        # "empresa",
        "poliza",
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Subir_programas_empresa.to_excel(
    "F:/Empresas SF/reportes/programas_empresa.xlsx", index=False
)
 """
#
Pacientes_con_id_final["esonoes1"] = Pacientes_con_id_final["rut"].isin(
    AccountContact_PacientesSF["Rut_y"]
)
Subir_AccountContact = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes1"] == False
]
Subir_AccountContact["ContactId"] = Subir_AccountContact["Id Contacto"]
Subir_AccountContact["AccountId"] = VempresaaAccount

Subir_AccountContact = Subir_AccountContact[
    [
        # "empresa",
        "poliza",
        "ContactId",
        "AccountId",
    ]
]
Subir_AccountContact.to_excel(
    "F:/Empresas SF/reportes/account_Contact.xlsx", index=False
)
if(campaña == "MetLife"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Empresa"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]
    Pacientes_con_id_final["llavepa"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]
    empresacontactoconvenio_PacientesSF["polizallavepa"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
elif(campaña == "Vida Security"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] +empresacontactoconvenio_PacientesSF["Rut Empresa"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutE"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"] 
    empresacontactoconvenio_PacientesSF["polizallavepa"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llavepa"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]

    
else:
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]    
    empresacontactoconvenio_PacientesSF["polizallavepa"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llavepa"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]




Pacientes_con_id_final["esonoes2"] = empresacontactoconvenio_PacientesSF["llave"].isin(
    Pacientes_con_id_final["llaveA"]
)
Pacientes_con_id_final["sionose"] = Pacientes_con_id_final["llavepa"].isin(
    empresacontactoconvenio_PacientesSF["polizallavepa"]
)

Subir_empresacontactoconvenio = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes2"] == False
]
Subir_empresacontactoconvenio2 = Pacientes_con_id_final[
    Pacientes_con_id_final["sionose"] == False
]
#Subir_empresacontactoconvenio = Pacientes_con_id_final

Subir_empresacontactoconvenio["Paciente__c"] = Subir_empresacontactoconvenio[
    "Id Account Paciente"
]

Subir_empresacontactoconvenio["N_Documento_Titular__c"] = Subir_empresacontactoconvenio[
    "rutT"
]
Subir_empresacontactoconvenio["Empresa__c"] = VempresaConvenio

Subir_empresacontactoconvenio = Subir_empresacontactoconvenio[
    [
        "empleador",
        "rutE",
        "poliza",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
        "esonoes2",
        #"llave",
        
    ]
]
Subir_empresacontactoconvenio.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio.xlsx", index=False)
Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final2.xlsx", index=False
    
)
Subir_empresacontactoconvenio2.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio2.xlsx", index=False)

001RN000006kbAtYAI 193331777
001RN000006kW9XYAU 201974755
001RN000006kcdDYAQ 154415513
001RN000006kcwbYAA 188486991
001RN000006kdpLYAQ 165578104
001RN000006kdXbYAI 154576517
001RN000006kZViYAM 177996335
001RN000006kcbZYAQ 190834212
001RN000006kPfwYAE 217273765
001RN000006keANYAY 201990513
001RN000006khMfYAI 170854012
001RN000006kgYgYAI 124868661
001RN000006kdB0YAI 12690777K
001RN000006ke2HYAQ 167432743
001RN000006kYbAYAU 165846192
001RN000006keLaYAI 179963477
001RN000006khOHYAY 180310401
001RN000006kgbuYAA 187583020
001RN000006kgvGYAQ 13412749K
001RN000006kZSPYA2 131911076
001RN000006khPtYAI 193041078
001RN000006khRVYAY 178578359
001RN000006keLbYAI 188619932
001RN000006kXaGYAU 167976735
001RN000006kbE4YAI 161364177
001RN000006kb1CYAQ 154795928
001RN000006khT7YAI 187740541
001RN000006khUjYAI 163324873
001RN000006kgP0YAI 181434295
001RN000006khXxYAI 178553305
001RN000006kdm8YAA 158486717
001RN000006khbBYAQ 167784496
001RN000006kZaSYAU 16551131K
001RN000006kfHfYAI 166117704
001RN000006kcR

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\4237995249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\4237995249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16032\4237995249.py:180: SettingWithCopyWarning: 
A value is try

Cuadrar eliminar pacientes q no estan en el listado
//-----------------------------------------------------------//


In [ ]:
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "163820064")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "153298939")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "191710940")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "182470082")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "HEGJ930928HMNRRL06")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16007258K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "86676540")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "202201466")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "105233469")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "130949185")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "255356798")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "34005945")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "185410072")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "197415029")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "254468134")].index, inplace=True)



empresacontactoconvenio_PacientesSF.head()

In [ ]:
empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/SFpacientes.xlsx", index=False)

In [ ]:
contactd = sf.Account.get("0018c00002lUyo0AAC")
contactd["PersonContactId"]

In [ ]:
# contactd["PersonContactId"]
print(datetime.now())

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/tod  es.xlsx", index=False)

query2 = "select P_liza**c, Nombre_de_Empresa**c, Rut_Empresa**c, Empresa_Padre**r.Name, Empresa**r.name,Holding**c, Plan**c, Pais**c, Activo**c, Id from Datos_Empresa**c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()


Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)


derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId\_\_pc": "derty", "Email": "example@example.com"})


Subir_pacientes.head()


Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account\_\_l.xlsx", index=False)


In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()